<a href="https://colab.research.google.com/github/AmirhosseinnnKhademi/GEN-AI---TF/blob/main/Course%204%20-%20Generative%20Deep%20Learning/W2/ungraded_labs/C4_W2_Lab_3_MNIST_DeepAutoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: MNIST Deep Autoencoder

Welcome back! In this lab, you will extend the shallow autoencoder you built in the previous exercise. The model here will have a deeper network so it can handle more complex images.

## Imports

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

import numpy as np
import matplotlib.pyplot as plt

## Prepare the Dataset

You will prepare the MNIST dataset just like in the previous lab.

In [ ]:
def map_image(image, label):
  '''Normalizes and flattens the image. Returns image as input and label.'''
  image = tf.cast(image, dtype=tf.float32) # Data conversion: This converts the image data to 32-bit floating point numbers. This is important because later operations (like division) require floating point precision.
  image = image / 255.0 # Normalization: Pixel values in images are usually in the range [0, 255]. Dividing by 255.0 scales them to the range [0, 1], which is common for neural network inputs.
  image = tf.reshape(image, shape=(784,)) # Flattening: This reshapes the image into a one-dimensional tensor with 784 elements. This is typical for a 28x28 image (since 28 * 28 = 784), effectively "flattening" the 2D image into a 1D vector.
  # MNIST has 28x28 images. 28x28 = 784
  return image, image

In [ ]:
# Load the train and test sets from TFDS

BATCH_SIZE = 128
SHUFFLE_BUFFER_SIZE = 1024

train_dataset = tfds.load('mnist', as_supervised=True, split="train") # The as_supervised=True parameter means that each example is returned as a tuple (image, label)
train_dataset = train_dataset.map(map_image) # The .shuffle() method randomizes the order of examples in the dataset.
train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE).repeat()

test_dataset = tfds.load('mnist', as_supervised=True, split="test")
test_dataset = test_dataset.map(map_image)
test_dataset = test_dataset.batch(BATCH_SIZE).repeat()

## Build the Model

As mentioned, you will have a deeper network for the autoencoder. Compare the layers here with that of the shallow network you built in the previous lab.

In [ ]:
def deep_autoencoder(inputs):
  '''Builds the encoder and decoder using Dense layers.'''
  encoder = tf.keras.layers.Dense(units=128, activation='relu')(inputs)
  encoder = tf.keras.layers.Dense(units=64, activation='relu')(encoder)
  encoder = tf.keras.layers.Dense(units=32, activation='relu')(encoder)

  decoder = tf.keras.layers.Dense(units=64, activation='relu')(encoder)
  decoder = tf.keras.layers.Dense(units=128, activation='relu')(decoder)
  decoder = tf.keras.layers.Dense(units=784, activation='sigmoid')(decoder)

  return encoder, decoder

# initialize/set the input tensor
inputs =  tf.keras.Input(shape=(784,))

# get the encoder and decoder output
deep_encoder_output, deep_autoencoder_output = deep_autoencoder(inputs)

# setup the encoder because you will visualize its output later
deep_encoder_model = tf.keras.Model(inputs=inputs, outputs=deep_encoder_output)

# setup the autoencoder
deep_autoencoder_model = tf.keras.Model(inputs=inputs, outputs=deep_autoencoder_output)

## Compile and Train the Model

In [ ]:
train_steps = 60000 // BATCH_SIZE
# The MNIST training dataset contains 60,000 images.
# Dividing 60,000 by the batch size (128) gives the number of batches needed for one epoch.
# The // operator performs floor division, ensuring an integer value (e.g., 60000 // 128 ≈ 468).

deep_autoencoder_model.compile(optimizer=tf.keras.optimizers.Adam(), loss='binary_crossentropy')
# We choose 'binary_crossentropy' because the autoencoder's output uses a sigmoid activation and the pixel values are normalized to the range [0, 1].
# In this setting, each pixel is treated as a probability, and binary crossentropy is effective
# at measuring the difference between the predicted probabilities (reconstructed pixels) and the true pixel values.
deep_auto_history = deep_autoencoder_model.fit(train_dataset, steps_per_epoch=train_steps, epochs=50)

## Display sample results

See the results using the model you just trained.

In [ ]:
def display_one_row(disp_images, offset, shape=(28, 28)):
  '''Display sample outputs in one row.'''
  for idx, test_image in enumerate(disp_images):
    plt.subplot(3, 10, offset + idx + 1)
    plt.xticks([])
    plt.yticks([])
    test_image = np.reshape(test_image, shape)
    plt.imshow(test_image, cmap='gray')


def display_results(disp_input_images, disp_encoded, disp_predicted, enc_shape=(8,4)):
  '''Displays the input, encoded, and decoded output values.'''
  plt.figure(figsize=(15, 5))
  display_one_row(disp_input_images, 0, shape=(28,28,))
  display_one_row(disp_encoded, 10, shape=enc_shape)
  display_one_row(disp_predicted, 20, shape=(28,28,))

In [ ]:
# take 1 batch of the dataset
test_dataset = test_dataset.take(1)
# The .take(1) method retrieves only one batch from the test dataset. Since the dataset was batched with a size of 128, this batch contains 128 examples.

# take the input images and put them in a list
# It extracts the original images from the batch so they can be later displayed and compared to the model’s outputs.
output_samples = []
for input_image, image in tfds.as_numpy(test_dataset):
      output_samples = input_image

# pick 10 random numbers to be used as indices to the list above
idxs = np.random.choice(BATCH_SIZE, size=10)

# get the encoder output
encoded_predicted = deep_encoder_model.predict(test_dataset)

# get a prediction for the test batch
deep_predicted = deep_autoencoder_model.predict(test_dataset)

# display the 10 samples, encodings and decoded values!
display_results(output_samples[idxs], encoded_predicted[idxs], deep_predicted[idxs])